In [1]:
%pylab inline
import psycopg2
from pyiem.network import Table as NetworkTable
import pandas as pd
from pyiem.plot import MapPlot
import numpy as np
from tqdm import tqdm
from pyiem.meteorology import windchill
from pyiem.datatypes import temperature, speed

Populating the interactive namespace from numpy and matplotlib


/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
ASOS = psycopg2.connect(database='asos', user='nobody', host='localhost', port=5555)
acursor = ASOS.cursor()

MESOSITE = psycopg2.connect(database='mesosite', user='nobody', host='localhost', port=5555)
mcursor = MESOSITE.cursor()

networks = ['AWOS',]
mcursor.execute("""SELECT distinct network from stations where country = 'US' and 
 network ~* 'ASOS' and state not in ('AK', 'HI') and length(id) = 3""")
for row in mcursor:
    networks.append( row[0] )

nt = NetworkTable(networks)
rows = []
sids = nt.sts.keys()
sids.sort()
for sid in tqdm(sids):
    if nt.sts[sid]['archive_begin'] is None or nt.sts[sid]['archive_begin'].year > 1973:
        continue

    # Figure out what our archive coverage is, 41 years would be perfect
    acursor.execute("""
    WITH obs as (
        SELECT date_trunc('hour', valid) as hr, tmpf, sknt from alldata
        where station = %s and valid > '1973-01-01' and tmpf is not null
        and sknt >= 0 and extract(month from valid) in (10, 11, 12, 1, 2, 3)
        and report_type = 2
    ), agg1 as (
        SELECT hr, tmpf, sknt, count(*) over (PARTITION by hr) from obs
    )
    SELECT hr, tmpf, sknt from agg1 where count = 1
    """, (sid,) )

    w = []
    for row in acursor:
        w.append(windchill(temperature(row[1], 'F'), speed(row[2], 'KT')).value('F'))
    if len(w) < 100:
        continue
    # 12 coldest hours over 6 winter months is 99.8th percentile
    val = np.percentile(np.array(w), np.arange(0, 1.5, 0.1))
    rows.append(dict(sid=sid, lat=nt.sts[sid]['lat'], lon=nt.sts[sid]['lon'],
                     val0=val[0], val1=val[1], val2=val[2], val3=val[3], val4=val[4],
                     val5=val[5], val6=val[6], val7=val[7], val8=val[8], val9=val[9],
                     val10=val[10], val11=val[11], val12=val[12], val13=val[13],
                    obs=len(w), network=nt.sts[sid]['network']))
    # print sid, len(w), val

df = pd.DataFrame(rows)
df.set_index('sid', inplace=True)
df['useme'] = True

100%|██████████| 2385/2385 [1:04:07<00:00,  3.04s/it]


In [7]:
df.to_csv('windchill_percentiles.csv')
df.at['3A1', 'useme'] = False
df.at['87Q', 'useme'] = False
print df[df['network'] == 'AL_ASOS'].sort_values('val0')


          lat       lon  network     obs       val0       val1      val10  \
sid                                                                         
DHN  31.32000 -85.45000  AL_ASOS  169780 -68.090892   7.841168  20.176041   
79J  31.30875 -86.39378  AL_ASOS   54480 -26.445132   2.696331  18.084801   
HSV  34.64389 -86.78611  AL_ASOS  184356 -25.003046  -7.597758   8.259502   
MSL  34.74385 -87.59962  AL_ASOS  169707 -22.907461  -6.438875   9.724472   
BHM  33.56546 -86.74488  AL_ASOS  179233 -22.494960  -1.895847  12.319945   
ANB  33.58817 -85.85811  AL_ASOS  170668 -22.007657   0.302002  14.133991   
MGM  32.30064 -86.39397  AL_ASOS  184543 -17.514402   5.897717  18.337974   
TCL  33.21190 -87.61610  AL_ASOS  171710 -14.797734   2.273748  16.141860   
MOB  30.68833 -88.24556  AL_ASOS  182933 -12.424623   7.491762  20.157599   
TOI  31.86042 -86.01214  AL_ASOS   71749 -10.949120   9.581153  20.176041   
GAD  33.97265 -86.08908  AL_ASOS   24239 -10.252605   0.014099  12.146652   

In [8]:
df2 = df[df['useme'] == True]
for p in range(14):
    percentile = 100. - (p * 0.1)
    values = df2['val%s' % (p,)]
    hours = 182. * 24. * ((100. - percentile) / 100.)
    m = MapPlot(sector='conus',
                title=("%.1fth Percentile Coldest Windchill Temperature (October thru March)"
                       ) % (percentile,),
                subtitle=("based on longterm automated stations hourly METARs, "
                          "%.1f is ~%.0f hours out of six months"
                         ) % (percentile, hours))
    m.contourf(np.array(df2.lon), np.array(df2.lat), np.array(values), np.arange(-50, 40, 5), units='F',
              cmap=plt.get_cmap('jet'))
    m.plot_values(df2.lon, df2.lat, values, fmt='%.0f', labelbuffer=15)
    m.postprocess(filename='windchill_%.0f.png' % (percentile * 10,))
    m.close()